# **Práctica 1**: Detección de lenguaje abusivo en Twitter




Este tutorial muestra un modelo basado en machine learning que permite la detección del lenguaje abusivo en Twitter, tomando como base los datos recogidos por Waseem and Hovy (2016). 

In [ ]:
#opcional: puedes subir los datos o montar tu Drive
#from google.colab import drive
#drive.mount('/content/drive')

Importamos algunos de los paquetes necesarios para este tutorial. Utilizaremos la libreria de módulos de Python **SciKitLearn**. 

In [2]:
import numpy as np
import pandas as pd 

from collections import Counter
import matplotlib.pyplot as plt

!pip install sklearn
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

In [4]:
# Tal como lo hicimos la clase pasada, podemos instalar e importar el módulo de preprocesamiento para tuits.
!pip install nlpretext
from nlpretext.social.preprocess import remove_emoji
from nlpretext.social.preprocess import remove_hashtag
from nlpretext.social.preprocess import remove_mentions
from nlpretext.basic.preprocess import replace_urls

     |████████████████████████████████| 92 kB 227 kB/s 
     |████████████████████████████████| 376 kB 29.5 MB/s 
     |████████████████████████████████| 654 kB 41.1 MB/s 
     |████████████████████████████████| 10.4 MB 2.7 MB/s 
     |████████████████████████████████| 1.5 MB 44.4 MB/s 
     |████████████████████████████████| 118 kB 55.5 MB/s 
     |████████████████████████████████| 2.6 MB 40.7 MB/s 
     |████████████████████████████████| 185 kB 41.5 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 6.8 MB 43.7 MB/s 
     |████████████████████████████████| 1.0 MB 42.3 MB/s 
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9309 sha256=976182c1097249f38fad0da28cc7a1941c97e735f58c5386c8a2bfd19d64432f
  Stored in directory: /root/.cache/pip/wheels/cb/19/58/4e8fdd0009a7f89dbce3c18fff2e0d0fa201d5cdfd16f113b7
  Created wheel for mosestokenizer: filename=mosestokenizer-1.1.0-py3-none-any.whl size=49119 sha256=b

In [ ]:
# opcional: aquí verificamos el directorio en el que estamos
import os
os.getcwd()

'/content'

Asignamos variables para los dos sets, train y test. Los datos para este tutorial están divididos en varios archivos de texto.

In [ ]:
import pandas as pd
import numpy as np

data_tweets_train = open('/content/drive/MyDrive/ML_Florida_Tutorial/Data/Waseem/waseemtrain.txt').read()
data_labels_train = open('/content/drive/MyDrive/ML_Florida_Tutorial/Data/Waseem/waseemtrainGold.txt').read()

data_tweets_test = open('/content/drive/MyDrive/ML_Florida_Tutorial/Data/Waseem/waseemtest.txt').read()
data_labels_test = open('/content/drive/MyDrive/ML_Florida_Tutorial/Data/Waseem/waseemtestGold.txt').read()

Limpiamos los datos quitando lo que no nos interese, como emojis, hashtags, urls... Podemos utilizar nuestra función de la clase pasada o hacerlo manualmente mediante *for loops* como aquí.

In [ ]:
tweets_train = []
labels_train = []

tweets_test = []
labels_test = []

for line in data_tweets_train.split("\n"):
    line = remove_emoji(line)
    line = remove_hashtag(line)
    line = remove_mentions(line)
    line = replace_urls(line, "")
    tweets_train.append(line)
    
for label in data_labels_train.split("\n"):
    labels_train.append(label)
    

for line in data_tweets_test.split("\n"):
    line = remove_emoji(line)
    line = remove_hashtag(line)
    line = remove_mentions(line)
    line = replace_urls(line, "")
    tweets_test.append(line)
    
    
for label in data_labels_test.split("\n"):
    labels_test.append(label)

Aquí tenemos algunos ejemplos para ambos sets:

In [ ]:
# 1 = abusive; 2 = non-abusive
print(tweets_train[23], labels_train[23])
print(tweets_train[12394], labels_train[12394])

Come and get your Jizya scumbag. I have it waiting in 0.4 cal copper and lead. 1
Read about the Muslim invasion of India from historian Will Durant. 2


In [ ]:
# 1 = abusive; 2 = non-abusive
print(tweets_test[23], labels_test[23])
print(tweets_test[754], labels_test[754])

Can we kill for Don? And does he give us a bunch of virgins in heaven for doing it? 1
RT : Bianca is feeling sick. She tried the baked Greek eggs. 2


Ahora crearemos un dataframe con los datos que hemos limpiado usando Pandas. 

In [ ]:
corpus = pd.DataFrame()
corpus['tweet'] = tweets_train
corpus['label'] = labels_train

Como vemos, el corpus está compuesto por 14143 filas (todos los tweets) y dos columnas (texts y labels).

In [ ]:
print('Shape of train set {}'.format(corpus.shape))

Shape of train set (14143, 2)


Este es el balance con respecto a las labels. 

In [ ]:
corpus['label'].value_counts()

2    9683
1    4460
Name: label, dtype: int64


---



Ahora que tenemos nuestros datos limpios y ordenados en un dataframe, podemos empezar a experimentar con diferentes métodos de modelaje estadístico. En este tutorial, usaremos solo un modelo (Support Vector Classification o SVC), pero jugaremos un poco con los features (variables o labels) y parameters (optimización del módelo). También puedes intentar probar con otros modelos estadísticos, como Naive Bayes, Random Forest, etc. (https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)

Parametros: **Unigrams**

Vamos a empezar usando unigrams para nuestro modelo. 

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

scikit-learn solo funciona con vectores, no con palabras, por lo que primero necesitamos convertir nuestros datos en vectores para después entrenar el modelo. 

In [ ]:
svc = Pipeline([
        ("count_vectorizer", CountVectorizer(analyzer = 'word', 
                                             ngram_range = (1,1),
                                             token_pattern=r'\b\w+\b', 
                                             min_df=1)),
        ("linear svc", SVC(kernel="rbf")) # try with "linear", "poly", "rbf" or "sigmoid"
    ])

In [ ]:
model = svc
model.probability = True
model.fit(tweets_train, labels_train)
y_pred = model.predict(tweets_test)
print(accuracy_score(labels_test, y_pred))
print(classification_report(labels_test, y_pred))

0.8150031786395423
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           1       0.79      0.56      0.66       496
           2       0.82      0.93      0.87      1076

    accuracy                           0.82      1573
   macro avg       0.54      0.50      0.51      1573
weighted avg       0.81      0.82      0.80      1573



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  


¿Cómo leer la tabla?   
**Precision**: the ratio of correctly predicted positive observations to the total predicted positive observations.   
**Recall**: the ratio of correctly predicted positive observations to the all observations in actual class - yes.   
**F1-score**: the weighted average of Precision and Recall.    

También podemos usar oraciones para ver la predicción asignada por el modelo. 

In [ ]:
sentence = (['Islam is worse than the Nazi party ever was.'])
label_prediction = model.predict(sentence)
print(label_prediction) 

['1']


In [ ]:
sentence = (['Islamic State Executes 5 Men In Mosul After Their Wives Fail To Wear New “Afghan-Style” Hijab…'])
label_prediction = model.predict(sentence)
print(label_prediction) 

['2']


---

Ahora, para intentar mejorar nuestro modelo, podemos intentar con bigrams:

Parametros: **Bigrams**


In [ ]:
svc = Pipeline([
        ("count_vectorizer", CountVectorizer(analyzer = 'word',
                                             ngram_range=(2, 2), 
                                             token_pattern=r'\b\w+\b', 
                                             min_df=1)),
        ("linear svc", SVC(kernel="rbf")) # try with "linear", "poly", "rbf" or "sigmoid"
    ])

In [ ]:
model = svc
model.probability = True
model.fit(tweets_train, labels_train)
y_pred = model.predict(tweets_test)
print(accuracy_score(labels_test, y_pred))
print(classification_report(labels_test, y_pred))

0.7768595041322314
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           1       0.87      0.35      0.50       496
           2       0.76      0.98      0.86      1076

    accuracy                           0.78      1573
   macro avg       0.54      0.44      0.45      1573
weighted avg       0.80      0.78      0.74      1573



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  


La precisión en general ha bajado. Probemos con un modelo de trigrams:

Parametros: **Trigrams**

In [ ]:
svc = Pipeline([
        ("count_vectorizer", CountVectorizer(analyzer = 'word',
                                             ngram_range=(3, 3), 
                                             token_pattern=r'\b\w+\b', 
                                             min_df=1)),
        ("linear svc", SVC(kernel="rbf")) # try with "linear", "poly", "rbf" or "sigmoid"
    ])

In [ ]:
model = svc
model.probability = True
model.fit(tweets_train, labels_train)
y_pred = model.predict(tweets_test)
print(accuracy_score(labels_test, y_pred))
print(classification_report(labels_test, y_pred))

0.7457088366179275
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           1       0.88      0.23      0.36       496
           2       0.73      0.99      0.84      1076

    accuracy                           0.75      1573
   macro avg       0.54      0.40      0.40      1573
weighted avg       0.78      0.75      0.69      1573



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  


Tampoco nos deja muy buenos resultados en comparacion con el modelo de unigrams. Intentemos con uno basado en caracteres:

Parametro: **Character-gram**

In [ ]:
svc = Pipeline([
        ("count_vectorizer", CountVectorizer(analyzer = 'char',
                                             ngram_range=(3, 5))),
        ("linear svc", SVC(kernel="rbf")) # try with "linear", "poly", "rbf" or "sigmoid"
    ])

In [ ]:
model = svc
model.probability = True
model.fit(tweets_train, labels_train)
y_pred = model.predict(tweets_test)
print(accuracy_score(labels_test, y_pred))
print(classification_report(labels_test, y_pred))